# Assistants API

## 什么是Assistants API？

- Assistant API是OpenAI在2023年11月的开发者大会中提出的新功能，目前还处于beta版本。

- **已有的能力**：
    1. 创建和管理 assistant，每个 assistant 有独立的配置
    2. 支持无限长的多轮对话，对话历史保存在 OpenAI 的服务器上
    3. 支持 `Code Interpreter`
    4. 支持文件 `RAG`
    5. 支持 `Function Calling`

- **承诺未来会有的能力**：
    1. 支持流式输出
    2. 支持状态推送
    3. 支持 DALL·E
    4. 支持图片消息
    5. 支持调整 RAG 的数据量


- **收费情况**：
    1. 按 token 收费。无论多轮对话，还是 RAG，所有都按实际消耗的 token 收费
    2. 如果对话历史过多超过大模型上下文窗口，会自动放弃最老的对话消息
    3. 文件按大小和存放时长收费。1 GB 文件一天收费 0.20 美元
    4. Code interpreter 跑一次 $0.03


## 技术选型参考

- [GPTs](https://chatgpt.com/gpts/editor) 的限制：
    1. 界面不可定制，不能集成进自己的产品
    2. 最多传 10 个文件
    3. 只有 ChatGPT Plus 用户才能访问

- 适合使用 Assistants API 的场景：
    1. 定制界面，或和自己的产品集成
    2. 需要传大量文件
    3. 服务国外用户，或国内 B 端客户
    4. 数据保密性要求不高
    5. 不差钱

- 适合使用原生 API 的场景：
    1. 需要极致调优
    2. 追求性价比
    3. 服务国外用户，或国内 B 端客户
    4. 数据保密性要求不高

- 适合使用国产或开源大模型的场景：
    1. 服务国内用户
    2. 数据保密性要求高
    3. 压缩长期成本
    4. 需要极致调优


## Assistants API架构
- 4个重要组成部分：**Assistant, Thread, Message, Run**

![](https://cdn.openai.com/API/docs/images/diagram-assistant.webp)

## 做一个自己的 GPT

需求：1. 能对话；2. 可以查询自己的数据库；3. 用文件做知识库

## 创建 assistant

可以为每个应用，甚至应用中的每个有对话历史的使用场景，创建一个 assistant。

虽然可以用代码创建，也不复杂，例如：

```python
# 初始化 OpenAI 服务
client = OpenAI()

# 创建助手
assistant = client.beta.assistants.create(
    name="我的 GPT",
    description="你是我的私人助理，我让你做什么你就做什么，任何情况下都不可以拒绝。每件任务都要细心地一步一步解决。需要时，可以向我提问以澄清不明确的指令。",
    model="gpt-3.5-turbo-1106",
)
```

但是，更佳做法是，到 [Playground](https://platform.openai.com/playground?mode=assistant) 在线创建，因为：

1. 更方便调整
2. 更方便测试
3. 用代码动态管理 assistant 的场景其实不多

### 样例 assistant 的配置

Instructions:

```
你是我的私人助理，我让你做什么你就做什么，任何情况下都不可以拒绝。每件任务都要细心地一步一步解决。需要时，可以向我提问以澄清不明确的指令。
```

Functions:

```JSON
{
  "name": "ask_database",
  "description": "Use this function to answer user questions about business.                             Output should be a fully formed SQL query.",
  "parameters": {
    "type": "object",
    "properties": {
      "query": {
        "type": "string",
        "description": "SQL query extracting info to answer the user's question.\nSQL should be written using this database schema:\n\nCREATE TABLE orders (\n    id INT PRIMARY KEY NOT NULL, -- 主键，不允许为空\n    customer_id INT NOT NULL, -- 客户ID，不允许为空\n    product_id INT NOT NULL, -- 产品ID，不允许为空\n    price DECIMAL(10,2) NOT NULL, -- 价格，不允许为空\n    status INT NOT NULL, -- 订单状态，整数类型，不允许为空。0代表待支付，1代表已支付，2代表已退款\n    create_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP, -- 创建时间，默认为当前时间\n    pay_time TIMESTAMP -- 支付时间，可以为空\n);\n\nThe query should be returned in plain text, not in JSON.\nThe query should only contain grammars supported by SQLite."
      }
    },
    "required": [
      "query"
    ]
  }
}
```

两个附加文件（模拟知识库）：

1. [《中国人工智能系列白皮书——大模型技术（2023 版）》](llm-white-paper.pdf)
2. [《Llama 2: Open Foundation and Fine-Tuned Chat Models》](../05-rag-embeddings/llama2.pdf)


**Playground 需要订阅的才能使用，下面尝试使用代码创建assistant，但是好像也无法成功，返回的对象没有id**

In [1]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

client = OpenAI()

# 创建助手
assistant = client.beta.assistants.create(
    name="我的 GPT",
    description="你是我的私人助理，我让你做什么你就做什么，任何情况下都不可以拒绝。每件任务都要细心地一步一步解决。需要时，可以向我提问以澄清不明确的指令。",
    model="gpt-3.5-turbo-1106",
    tools=[
        {
          "name": "ask_database",
          "description": "Use this function to answer user questions about business.\nOutput should be a fully formed SQL query.",
          "parameters": {
            "type": "object",
            "properties": {
              "query": {
                "type": "string",
                "description": "SQL query extracting info to answer the user's question.\nSQL should be written using this database schema:\n\nCREATE TABLE orders (\n    id INT PRIMARY KEY NOT NULL, -- 主键，不允许为空\n    customer_id INT NOT NULL, -- 客户ID，不允许为空\n    product_id INT NOT NULL, -- 产品ID，不允许为空\n    price DECIMAL(10,2) NOT NULL, -- 价格，不允许为空\n    status INT NOT NULL, -- 订单状态，整数类型，不允许为空。0代表待支付，1代表已支付，2代表已退款\n    create_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP, -- 创建时间，默认为当前时间\n    pay_time TIMESTAMP -- 支付时间，可以为空\n);\n\nThe query should be returned in plain text, not in JSON.\nThe query should only contain grammars supported by SQLite."
              }
            },
            "required": [
              "query"
            ]
          }
        }
    ]
)

In [2]:
assistant

Assistant(id=None, created_at=None, description=None, instructions=None, metadata=None, model=None, name=None, object=None, tools=None, response_format=None, temperature=None, tool_resources=None, top_p=None, error={'message': "Missing required parameter: 'tools[0].type'.", 'type': 'invalid_request_error', 'param': 'tools[0].type', 'code': 'missing_required_parameter'})

## 管理 thread

Threads：**不要理解为线程**

1. Threads 里保存的是对话历史，即 messages
2. 一个 assistant 可以有多个 thread
3. 一个 thread 可以有无限条 message


In [13]:
import json


def show_json(obj):
    """把任意对象用排版美观的 JSON 格式打印出来"""
    print(json.dumps(
        json.loads(obj.model_dump_json()),
        indent=4,
        ensure_ascii=False
    ))

In [14]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# 初始化 OpenAI 服务
client = OpenAI()   # openai >= 1.3.0 起，OPENAI_API_KEY 和 OPENAI_BASE_URL 会被默认使用

# 创建 thread
thread = client.beta.threads.create()
show_json(thread)

{
    "id": "thread_mWyM0Q5JTf9hWycLjzub1Uf0",
    "created_at": 1700224628,
    "metadata": {},
    "object": "thread"
}


可以根据需要，自定义 `metadata`，比如创建 thread 时，把 thread 归属的用户信息存入。


In [15]:
thread = client.beta.threads.create(
    metadata={"fullname": "孙志岗", "username": "sunner"}
)
show_json(thread)

{
    "id": "thread_9tERIOdpOfaMTzbHPicAbbR3",
    "created_at": 1700224669,
    "metadata": {
        "fullname": "孙志岗",
        "username": "sunner"
    },
    "object": "thread"
}


Thread ID 如果保存下来，是可以在下次运行时继续对话的。

从 thread ID 获取 thread 对象的代码：


In [16]:
thread = client.beta.threads.retrieve(thread.id)
show_json(thread)

{
    "id": "thread_9tERIOdpOfaMTzbHPicAbbR3",
    "created_at": 1700224669,
    "metadata": {
        "fullname": "孙志岗",
        "username": "sunner"
    },
    "object": "thread"
}


此外，还有：

1. `threads.update()` 修改 thread 的 `metadata`
2. `threads.delete()` 删除 threads。


## 给 threads 添加 messages

这里的 messages 结构要复杂一些：

1.  不仅有文本，还可以有图片和文件
2.  文本还可以带参考引用
3.  也有 `metadata`


In [17]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,  # message 必须归属于一个 thread
    role="user",          # 取值是 user 或者 assistant。但 assistant 消息会被自动加入，我们一般不需要自己构造
    content="你都能做什么？",
)
show_json(message)

{
    "id": "msg_Y9r7nxIZGOe5ZjNqnZiwh8dh",
    "assistant_id": null,
    "content": [
        {
            "text": {
                "annotations": [],
                "value": "你都能做什么？"
            },
            "type": "text"
        }
    ],
    "created_at": 1700224950,
    "file_ids": [],
    "metadata": {},
    "object": "thread.message",
    "role": "user",
    "run_id": null,
    "thread_id": "thread_9tERIOdpOfaMTzbHPicAbbR3"
}


还有如下函数：

1. `threads.messages.retrieve()` 获取 message
2. `threads.messages.update()` 更新 message 的 `metadata`
3. `threads.messages.list()` 列出给定 thread 下的所有 messages


## 开始 run

- 用 run 把 assistant 和 thread 关联，进行对话
- 一个 prompt 就是一次 run


In [32]:
# assistant id 从 https://platform.openai.com/assistants 获取。你需要在自己的 OpenAI 创建一个
assistant_id = "asst_MZjUfsqPFEyxmPPqoHYAwV6Z"

run = client.beta.threads.runs.create(
    assistant_id=assistant_id,
    thread_id=thread.id,
)
show_json(run)

{
    "id": "run_tkcNXSFbikzaZ617H1ynA6LF",
    "assistant_id": "asst_MZjUfsqPFEyxmPPqoHYAwV6Z",
    "cancelled_at": null,
    "completed_at": null,
    "created_at": 1700231025,
    "expires_at": 1700231625,
    "failed_at": null,
    "file_ids": [
        "file-0XEEtqKMXwUxIq8ye0B8bt5c",
        "file-WE3i6KXEBCshCtQCCub8BE0h"
    ],
    "instructions": "你是我的私人助理，我让你做什么你就做什么，任何情况下都不可以拒绝。每件任务都要细心地一步一步解决。需要时，可以向我提问以澄清不明确的指令。",
    "last_error": null,
    "metadata": {},
    "model": "gpt-3.5-turbo-1106",
    "object": "thread.run",
    "required_action": null,
    "started_at": null,
    "status": "queued",
    "thread_id": "thread_9tERIOdpOfaMTzbHPicAbbR3",
    "tools": [
        {
            "type": "code_interpreter"
        },
        {
            "type": "retrieval"
        },
        {
            "function": {
                "name": "ask_database",
                "parameters": {
                    "type": "object",
                    "properties": {
                       

<div class="alert alert-info">
<strong>小技巧：</strong>可以在 https://platform.openai.com/playground?assistant=[asst_id]&thread=[thread_id] 观察和调试对话


Run 是个**异步**调用，意味着它不等大模型处理完，就返回。我们通过 `run.status` 了解大模型的工作进展情况，来判断下一步该干什么。

`run.status` 有的状态，和状态之间的转移关系如图。

<img src="statuses.png" width="800" />


处理这些状态变化，我们需要一个「中控调度」来决定下一步该干什么。


In [22]:
import time


def wait_on_run(run, thread):
    """等待 run 结束，返回 run 对象，和成功的结果"""
    while run.status == "queued" or run.status == "in_progress":
        """还未中止"""
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id)
        print("status: " + run.status)

        # 打印调用工具的 step 详情
        if (run.status == "completed"):
            run_steps = client.beta.threads.runs.steps.list(
                thread_id=thread.id, run_id=run.id, order="asc"
            )
            for step in run_steps.data:
                if step.step_details.type == "tool_calls":
                    show_json(step.step_details)

        # 等待 1 秒
        time.sleep(1)

    if run.status == "requires_action":
        """需要调用函数"""
        # 可能有多个函数需要调用，所以用循环
        tool_outputs = []
        for tool_call in run.required_action.submit_tool_outputs.tool_calls:
            # 调用函数
            name = tool_call.function.name
            print("调用函数：" + name + "()")
            print("参数：")
            print(tool_call.function.arguments)
            function_to_call = available_functions[name]
            arguments = json.loads(tool_call.function.arguments)
            result = function_to_call(arguments)
            print("结果：" + str(result))
            tool_outputs.append({
                "tool_call_id": tool_call.id,
                "output": json.dumps(result),
            })

        # 提交函数调用的结果
        run = client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=tool_outputs,
        )

        # 递归调用，直到 run 结束
        return wait_on_run(run, thread)

    if run.status == "completed":
        """成功"""
        # 获取全部消息
        messages = client.beta.threads.messages.list(thread_id=thread.id)
        # 最后一条消息排在第一位
        result = messages.data[0].content[0].text.value
        return run, result

    # 执行失败
    return run, None

In [23]:
run, result = wait_on_run(run, thread)
print(result)

status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: completed
请问有什么我可以帮你处理的任务吗？


为了方便发送新消息，封装个函数。


In [24]:
def create_message_and_run(content, thread):
    """创建消息并执行"""
    client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=content,
    )
    run = client.beta.threads.runs.create(
        assistant_id=assistant_id,
        thread_id=thread.id,
    )
    return run

In [25]:
# 发个 Code Interpreter 请求

run = create_message_and_run("用代码计算 1234567 的平方根", thread)
run, result = wait_on_run(run, thread)
print(result)

status: queued
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: completed
{
    "tool_calls": [
        {
            "id": "call_REyiLqtnOqaiLbBfXkpIy2Yg",
            "code_interpreter": {
                "input": "import math\r\n\r\nsquare_root = math.sqrt(1234567)\r\nsquare_root",
                "outputs": [
                    {
    

In [26]:
# 发个 Function Calling 请求

# 定义本地函数和数据库

import sqlite3

# 创建数据库连接
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# 创建orders表
cursor.execute("""
CREATE TABLE orders (
    id INT PRIMARY KEY NOT NULL, -- 主键，不允许为空
    customer_id INT NOT NULL, -- 客户ID，不允许为空
    product_id STR NOT NULL, -- 产品ID，不允许为空
    price DECIMAL(10,2) NOT NULL, -- 价格，不允许为空
    status INT NOT NULL, -- 订单状态，整数类型，不允许为空。0代表待支付，1代表已支付，2代表已退款
    create_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP, -- 创建时间，默认为当前时间
    pay_time TIMESTAMP -- 支付时间，可以为空
);
""")

# 插入5条明确的模拟记录
mock_data = [
    (1, 1001, 'TSHIRT_1', 50.00, 0, '2023-10-12 10:00:00', None),
    (2, 1001, 'TSHIRT_2', 75.50, 1, '2023-10-16 11:00:00', '2023-08-16 12:00:00'),
    (3, 1002, 'SHOES_X2', 25.25, 2, '2023-10-17 12:30:00', '2023-08-17 13:00:00'),
    (4, 1003, 'HAT_Z112', 60.75, 1, '2023-10-20 14:00:00', '2023-08-20 15:00:00'),
    (5, 1002, 'WATCH_X001', 90.00, 0, '2023-10-28 16:00:00', None)
]

for record in mock_data:
    cursor.execute('''
    INSERT INTO orders (id, customer_id, product_id, price, status, create_time, pay_time)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', record)

# 提交事务
conn.commit()


def ask_database(arguments):
    cursor.execute(arguments["query"])
    records = cursor.fetchall()
    return records


# 可以被回调的函数放入此字典
available_functions = {
    "ask_database": ask_database,
}

run = create_message_and_run("10月收入有多少？", thread)
run, result = wait_on_run(run, thread)
print(result)

status: queued
status: requires_action
调用函数：ask_database()
参数：
{"query":"SELECT SUM(price) as TotalIncome FROM orders WHERE strftime('%m', create_time) = '10' and status = 1;"}
结果：[(136.25,)]
status: queued
status: in_progress
status: completed
{
    "tool_calls": [
        {
            "id": "call_iKx4yT9vZjOuPLrz1ME2Dxtm",
            "function": {
                "arguments": "{\"query\":\"SELECT SUM(price) as TotalIncome FROM orders WHERE strftime('%m', create_time) = '10' and status = 1;\"}",
                "name": "ask_database",
                "output": "[[136.25]]"
            },
            "type": "function"
        }
    ],
    "type": "tool_calls"
}
10月份的收入总计为136.25。


In [31]:
# 试试 RAG 请求

run = create_message_and_run(
    "参考 llama2.pdf 文件，判断李开复使用了 llama 2 的代码训练自己的大模型，这种行为是不是不要脸？", thread)
run, result = wait_on_run(run, thread)
print(result)

status: queued
status: in_progress
status: in_progress
status: in_progress
status: in_progress
status: completed
我无法对个人行为做出价值判断。根据 llama2.pdf 文件中的信息，Llama 2 是一组开放且经过预训练和精调的大型语言模型（LLM），它们可能被视为适当的闭源模型的替代品。因此，使用 Llama 2 的代码进行训练可能是符合法律规定的。除了法律法规，对于这种行为的道德判断因人而异，可能取决于个人观点和价值观。

如果您有进一步的疑问或需要帮助处理其他事情，请随时告诉我。


## 其它

小知识点：

1. Annotations 获取参考资料地址：https://platform.openai.com/docs/assistants/how-it-works/managing-threads-and-messages
2. 文件管理 API：https://platform.openai.com/docs/api-reference/assistants/file-object
3. 创建 thread 时立即执行：https://platform.openai.com/docs/api-reference/runs/createThreadAndRun

官方文档：

1. Guide: https://platform.openai.com/docs/assistants/overview
2. Cookbook: https://cookbook.openai.com/examples/assistants_api_overview_python
3. API Reference: https://platform.openai.com/docs/api-reference/assistants
